In [ ]:
# step 8
from mne.time_frequency import tfr_morlet
from alive_progress import alive_bar
import matplotlib.pyplot as plt
from tqdm import tqdm
import nibabel as nib
import pandas as pd
import numpy as np
import seaborn as sns
import scipy
import h5py
import mne
import sys
import os
import mantel

subject_num = 34
var_loc = 'E:/workspace/Trust_Data_and_Results/Haoming/varibility/result/varibilit_corr_20_exp2_new_all_step_corr/'
behaviorVar_loc = 'E:/workspace/Trust_Data_and_Results/Haoming/varibility/behavior_result/behavior_varibility.npy'
# eegVar_loc = 'E:/workspace/Trust_Data_and_Results/Haoming/varibility/result/varibilit_corr_20_exp2_new_all_step_corr/all_var_data_one_all_6band.npy'
result_loc = 'E:/workspace/Trust_Data_and_Results/Haoming/varibility/result/varibilit_corr_20_exp2_new_all_step_corr/corr_result/temporal_result_win_20/'
plot_loc = result_loc + 'plot_result_untrust/'

experiment_2_event = [1, 2, 3, 4]
experiment_3_event = [30, 31, 32, 33]
experiment_4_event = [30, 31, 32, 33]
# file_name = ['ERP', 'delta', 'theta', 'alpha', 'beta', 'gamma']
# # file_name = ['delta_theta_alpha']
# power_name = ['delta', 'theta', 'alpha']
power_name = ['alpha']
condition_name = ['0Trust', '1Trust', '2Trust', '3Trust']


In [ ]:
# The length of the time window in units
window_length = 50

# Start and end time of the data to be analyzed The first time is the start time and the second time is the end time  
involve_time = [-0.1, 0.7]

# Subjects with incomplete data were removed
bad_subject = np.array([3, 5, 15]) -1
all_subject = np.arange(0,34,1)

# Delete specific elements
good_subject = np.setdiff1d(all_subject, bad_subject)

In [ ]:
channel_location_dict = {"Fp1":	[-26.25, 83, -18.25],
"Fp2": [26.25, 83, -18.25],
"F3": [-48.75, 57.5, 36.5],
"F4": [48.75, 57.5, 36.5],
"C3": [-63.75, -13, 65.75],
"C4": [63.75, -13, 65.75],
"P3": [-48, -86.5, 53],
"P4": [48, -86.5, 53],
"O1": [-27, -118, 4.25],
"O2": [27, -118, 4.25],
"F7": [-69, 44, -18.25],
"F8": [69, 44, -18.25],
"T7": [-84, -18.25, -13],
"T8": [84, -18.25, -13],
"P7": [-69, -80.5, -4.75],
"P8": [69, -80.5, -4.75],
"Fz": [0, 63.5, 59],
"Cz": [0, -10, 99.5],
"Pz": [0, -88, 77],
"FC1": [-32.25, 28.25, 77.75],
"FC2": [32.25, 28.25, 77.75],
"CP1": [-35.25, -52, 90.5],
"CP2": [35.25, -52, 90.5],
"FC5": [-75.75, 20, 21.5],
"FC6": [75.75, 20, 21.5],
"CP5": [-78, -51.25, 31.25],
"CP6": [78, -51.25, 31.25],
"FT9": [-72.75, 8.75, -55.75],
"FT10": [72.75, 8.75, -55.75],
"TP9": [-71.25, -49, -49.75],
"TP10": [71.25, -49, -49.75],
"F1": [-25.5, 62, 53.75],
"F2": [25.5, 62, 53.75],
"C1": [-35.25, -11.5, 89.75],
"C2": [35.25, -11.5, 89.75],
"P1": [-27, -88, 70.25],
"P2": [27, -88, 70.25],
"AF3": [-32.25, 80.75, 11.75],
"AF4": [32.25, 80.75, 11.75],
"FC3": [-59.25, 25.25, 54.5],
"FC4": [59.25, 25.25, 54.5],
"CP3": [-63, -52, 66.5],
"CP4": [63, -52, 66.5],
"PO3": [-31.5, -109.75, 32.75],
"PO4": [31.5, -109.75, 32.75],
"F5": [-63, 51.5, 11.75],
"F6": [63, 51.5, 11.75],
"C5": [-81.75, -14.5, 29],
"C6": [81.75, -14.5, 29],
"P5": [-63.75, -83.5, 26.75],
"P6": [63.75, -83.5, 26.75],
"AF7": [-50.25, 68, -19],
"AF8": [50.25, 68, -19],
"FT7": [-79.5, 15.5, -16],
"FT8": [79.5, 15.5, -16],
"TP7": [-80.25, -49.75, -8.5],
"TP8": [80.25, -49.75, -8.5],
"PO7": [-50.25, -103, 0.5],
"PO8": [50.25, -103, 0.5],
"Fpz": [0, 83, -18.25],
"CPz": [0, -53.5, 98],
"POz": [0, -109, 44],
"Oz": [0, -122.5, 8]}
# ['Fp1', 'Fp2', 'Fz', 'Cz', 'Pz', 'Fpz', 'CPz', 'POz', 'Oz'].
mymontage = mne.channels.make_dig_montage(ch_pos=channel_location_dict)#, nasion=[0, -10, 99.5], lpa=[-71.25, -49, -49.75], 
#                                        rpa=[71.25, -49, -49.75])

In [ ]:
behavior_varibility = np.load(behaviorVar_loc)
eeg_varibility = np.load(var_loc + 'all_var_data_nor.npy', allow_pickle=True).item()
print(behavior_varibility.shape)
print(eeg_varibility['data'].shape[-2])
print(eeg_varibility.keys())
print(eeg_varibility['event_name'])
# print(behavior_varibility)
# eeg_varibility['data'] = eeg_varibility['data'][good_subject]
behavior_varibility = behavior_varibility[good_subject]
print(eeg_varibility['data'].shape)
print(behavior_varibility.shape)

In [ ]:


# 通过对比involve_time和epoch的时间序列来找到是第几位数字开始计算相关性
# iter condition, iter power, iter channel, iter time
time_series = np.arange(-0.2, 1, 0.001)
time_series = np.around(time_series,3)
start_time = np.where(time_series == involve_time[0])[0][0]
end_time = np.where(time_series == involve_time[1])[0][0]
iter_condition = 3
iter_power = 0
figure_result_loc = 'E:/workspace/Trust_Data_and_Results/Haoming/varibility/result/varibilit_corr_20_exp2_new_all_step_corr/corr_result/temporal_figure_pearson_permu/'
# Generates an empty result array
iter_condition_and_power_result = []
for iter_elec in tqdm(range(eeg_varibility['data'].shape[-2])):
    # Generates an empty result array for each electrode
    iter_elec_result = []
    # Calculate the variation once for each time window
    for iter_window in range(end_time - start_time):# 步长为1
        # Get the data for each time window  
        t_begin = start_time + iter_window - round(window_length/2)
        t_end = start_time + iter_window + round(window_length/2)

        iter_eeg_varibility_data = eeg_varibility['data'][:,iter_power, iter_condition, iter_elec, t_begin:t_end]
        iter_eeg_varibility_data_mean = np.mean(iter_eeg_varibility_data, axis=-1)

        # Extract behavioral data to X and eeg variation to Y
        x = behavior_varibility[:, iter_condition]
        y = iter_eeg_varibility_data_mean

        # Calculate correlations and statistical significance, and store results
        scipy_corr = scipy.stats.pearsonr(x, y)
        # scipy_corr = scipy.stats.spearmanr(x, y)
        # scipy_corr = mantel.test(x, y, perms=5000,method='pearson')

        # Store results to results for each electrode
        if (scipy_corr[1] <= 0.05) & (scipy_corr[0] > 0):
            iter_elec_result.append(1)
        elif (scipy_corr[1] <= 0.05) & (scipy_corr[0] < 0):
            iter_elec_result.append(-1)
        else:
            iter_elec_result.append(0)

    # Store results to the total result matrix
    iter_condition_and_power_result.append(iter_elec_result)

iter_condition_and_power_result= np.array(iter_condition_and_power_result)
# create a new epochs info
info = mne.create_info(
    ch_names = eeg_varibility['ch_names'],
    ch_types = 'eeg',
    sfreq = 1000)

# create a new ROI based epochs
significiant_evoked = mne.EvokedArray(data = iter_condition_and_power_result, 
    info = info, tmin=-0.1)
# Set electrode position coordinates
significiant_evoked.set_montage(mymontage)
# plt.close('all')
#plt.rcParams['figure.figsize'] = (20,20) # 设置figure_size尺寸
all_time = np.arange(-0.1, 0.35, 0.002)
all_time = np.around(all_time, 3)
for iter_times in range(len(all_time)):

    times = all_time[iter_times]
    fig = significiant_evoked.plot_topomap(outlines='head',times=times, size=5, vmin=-1500000, vmax=1500000, show_names=False, sensors='kv', extrapolate='head', cmap='RdBu_r', scalings=None, colorbar=False)# cmap GnBu

    # save the figure
    iter_figure_result_loc = figure_result_loc + condition_name[iter_condition] + '/' + power_name[iter_power] + '/'
    if not os.path.exists(iter_figure_result_loc):
        os.makedirs(iter_figure_result_loc)

    fig.savefig(iter_figure_result_loc + str(iter_times) + '_' + str(times) + '_pic.png')


# all_data.shape